In [1]:
from PIL import Image, ImageOps
import numpy, os
from sklearn.ensemble import AdaBoostClassifier
from sklearn.cross_validation import cross_val_score
import numpy as np
import pandas as pd

/home/asus/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
path="dataset/"
Xlist=[]
Ylist=[]
size = 100, 100

In [3]:
for directory in os.listdir(path):
    for file in os.listdir(path+directory):
        print(path+directory+"/"+file)
        img=Image.open(path+directory+"/"+file)
        #resize
        thumb = ImageOps.fit(img, size, Image.ANTIALIAS)
        image_data = np.array(thumb).flatten()[:100]
        #image_data=numpy.array(img).flatten()[:50] #in my case the images dont have the same dimensions, so [:50] only takes the first 50 values
        Xlist.append(image_data)
        Ylist.append(directory)

dataset/object/27.png
dataset/object/82.png
dataset/object/83.png
dataset/object/100.png
dataset/object/0.png
dataset/object/13.png
dataset/object/45.png
dataset/object/64.png
dataset/object/19.png
dataset/object/101.png
dataset/object/40.png
dataset/object/97.png
dataset/object/41.png
dataset/object/7.png
dataset/object/66.png
dataset/object/55.png
dataset/object/56.png
dataset/object/65.png
dataset/object/18.png
dataset/object/24.png
dataset/object/105.png
dataset/object/116.png
dataset/object/117.png
dataset/object/104.png
dataset/object/63.png
dataset/object/38.png
dataset/object/58.png
dataset/object/103.png
dataset/object/112.png
dataset/object/33.png
dataset/object/76.png
dataset/object/59.png
dataset/object/96.png
dataset/object/91.png
dataset/object/57.png
dataset/object/2.png
dataset/object/75.png
dataset/object/107.png
dataset/object/50.png
dataset/object/16.png
dataset/object/32.png
dataset/object/15.png
dataset/object/5.png
dataset/object/72.png
dataset/object/52.png
datas

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Xlist, Ylist, test_size=0.2)

### AdaBoostClassifier

In [5]:
clf=AdaBoostClassifier(n_estimators=100)

In [6]:
scores = cross_val_score(clf, X_train, y_train, cv=3)

In [7]:
print(scores.mean())

0.77037037037


### GaussianNB

In [8]:
from sklearn.naive_bayes import GaussianNB

In [9]:
clf = GaussianNB()

In [10]:
scores = cross_val_score(clf, Xlist, Ylist)

In [11]:
print(scores.mean())

0.721908939014


### KNeighborsClassifier

In [12]:
from sklearn.neighbors import KNeighborsClassifier

In [13]:
clf = KNeighborsClassifier(n_neighbors=10)

In [14]:
scores = cross_val_score(clf, Xlist, Ylist)

In [15]:
print(scores.mean())

0.751357560568


### LinearSVC

In [16]:
from sklearn.svm import LinearSVC

In [17]:
clf = LinearSVC()

In [18]:
scores = cross_val_score(clf, Xlist, Ylist)

In [19]:
print(scores.mean())

0.638575605681


### SVC

In [20]:
from sklearn.svm import SVC

In [21]:
clf = SVC()

In [22]:
scores = cross_val_score(clf, Xlist, Ylist)

In [23]:
print(scores.mean())

0.668650793651


### GaussianProcessClassifier

In [24]:
from sklearn.gaussian_process import GaussianProcessClassifier

In [25]:
clf = GaussianProcessClassifier()

In [26]:
scores = cross_val_score(clf, Xlist, Ylist)

In [27]:
print(scores.mean())

0.491228070175


### RandomForestClassifier

In [28]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
clf = RandomForestClassifier()

In [30]:
scores = cross_val_score(clf, Xlist, Ylist)

In [31]:
print(scores.mean())

0.710317460317


# Hyperparameters Tuning using sklearn pipeline and gridsearch

In [32]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [33]:
pipe = [Pipeline([
        ('clf', RandomForestClassifier()),
    ]),
    Pipeline([
        ('clf', KNeighborsClassifier()),
    ]),
    Pipeline([
        ('clf', GaussianProcessClassifier()),
    ]),
    Pipeline([
        ('clf', AdaBoostClassifier()),
    ]),
    Pipeline([
        ('clf', SVC()),
    ]),
]

In [34]:
param_grid = [dict(clf__n_estimators=[3, 10, 100]),
              dict(clf__n_neighbors=[3,10]),
              dict(clf__n_restarts_optimizer=[0,1]),
              dict(clf__n_estimators=[3, 10, 100]),
              dict(clf__C=[3, 10, 100]),
             ]

In [35]:
grid_search = GridSearchCV(pipe, param_grid=param_grid, n_jobs=-1, verbose=1, cv=3)

In [36]:
#grid_search.fit(Xlist, Ylist)

In [37]:
# Utility function to report best scores
def report(results, n_top=10):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [38]:
from time import time

for i in range(len(pipe)):
        start = time()
        print("-----")
        print("classifier:")
        print(pipe[i].named_steps['clf'])
        grid_search = GridSearchCV(pipe[i], param_grid[i], n_jobs=-1, verbose=1, cv=3)
        grid_search.fit(X_train, y_train)
        print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
              % (time() - start, len(grid_search.cv_results_['params'])))
        print("finished GridSearch")
        report(grid_search.cv_results_)

-----
classifier:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    1.5s finished


GridSearchCV took 2.38 seconds for 3 candidate parameter settings.
finished GridSearch
Model with rank: 1
Mean validation score: 0.815 (std: 0.073)
Parameters: {'clf__n_estimators': 100}

Model with rank: 2
Mean validation score: 0.763 (std: 0.093)
Parameters: {'clf__n_estimators': 10}

Model with rank: 3
Mean validation score: 0.756 (std: 0.110)
Parameters: {'clf__n_estimators': 3}

-----
classifier:
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
Fitting 3 folds for each of 2 candidates, totalling 6 fits
GridSearchCV took 0.23 seconds for 2 candidate parameter settings.
finished GridSearch
Model with rank: 1
Mean validation score: 0.778 (std: 0.048)
Parameters: {'clf__n_neighbors': 3}

Model with rank: 2
Mean validation score: 0.704 (std: 0.010)
Parameters: {'clf__n_neighbors': 10}

-----
classifier:
GaussianProcessClassifier(copy_X_train=True, kernel=None,
            

[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.1s finished


GridSearchCV took 0.36 seconds for 2 candidate parameter settings.
finished GridSearch
Model with rank: 1
Mean validation score: 0.489 (std: 0.000)
Parameters: {'clf__n_restarts_optimizer': 0}

Model with rank: 1
Mean validation score: 0.489 (std: 0.000)
Parameters: {'clf__n_restarts_optimizer': 1}

-----
classifier:
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)
Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.9s finished


GridSearchCV took 1.16 seconds for 3 candidate parameter settings.
finished GridSearch
Model with rank: 1
Mean validation score: 0.807 (std: 0.093)
Parameters: {'clf__n_estimators': 3}

Model with rank: 2
Mean validation score: 0.756 (std: 0.048)
Parameters: {'clf__n_estimators': 100}

Model with rank: 3
Mean validation score: 0.733 (std: 0.054)
Parameters: {'clf__n_estimators': 10}

-----
classifier:
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Fitting 3 folds for each of 3 candidates, totalling 9 fits
GridSearchCV took 0.35 seconds for 3 candidate parameter settings.
finished GridSearch
Model with rank: 1
Mean validation score: 0.689 (std: 0.031)
Parameters: {'clf__C': 3}

Model with rank: 1
Mean validation score: 0.689 (std: 0.031)
Parameters: {'clf__C': 10}

Model with rank: 1
Mean validation score: 0.689

[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:    0.1s finished
